In [1]:
import weaviate
import weaviate.classes as wvc
import os
import requests
import json
from dotenv import load_dotenv
from weaviate.classes.config import Configure, Property, DataType, Tokenization
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter

load_dotenv()
openai_api_key = os.getenv("OPENAI_KEY")

client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=weaviate.config.AdditionalConfig(timeout=(10, 25)),
    headers={
        "X-OpenAI-Api-Key": openai_api_key  # Replace with your inference API key
    }
)

In [2]:
path_to_pdf = 'pdf_docs'
os.listdir(path_to_pdf)

['Claiming a social welfare payment.pdf',
 'Pregnancy and social welfare payments.pdf']

In [3]:
path_to_pdf = 'pdf_docs'

documents_text = []

for doc in os.listdir(path_to_pdf):

    doc_path = f'{path_to_pdf}/{doc}'
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
    docs = text_splitter.split_documents(pages)
    documents_text.append(docs)

documents_text = [item for sublist in documents_text for item in sublist]

In [4]:
for d in documents_text:
    print(d.metadata)
    print(d.page_content)
    print(d.metadata['source'])
    print(d.metadata['page'])
    break

{'source': 'pdf_docs/Claiming a social welfare payment.pdf', 'page': 0}
4/16/24, 10:04 PM Claiming a social welfare payment
https://www.citizensinformation.ie/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/applying-for-a-social-welfare-p… 1/4Claiming a social welfare payment
Introduc on
Rules
How to apply
Where to apply
Further informa on
Introduc on
The Department of Social Protec on has a range of social welfare payments that provide ﬁnancial
support. To get a social welfare payment, you must apply for it. So, if you think you are en tled to a
payment you should always apply.
Your ﬁrst step before applying, is to ﬁnd out which payment you may be en tled to.
Payments are available for:
People who are unemployed
Families and children
Widows, widowers and surviving civil partners
Guardians or orphans
Older and re red people
People with a disability
Carers
If you get a social welfare payment you may also qualify for extra social welfare beneﬁts.
Household

In [7]:
client.collections.delete("citizens_info_docs") 
client.collections.create(
    "citizens_info_docs",

    properties=[  
        Property(
            name="source",
            data_type=DataType.TEXT,

        ),
        Property(
            name="page", 
            data_type=DataType.TEXT
        ),

        Property(
            name="content", 
            data_type=DataType.TEXT,
            vectorize_property_name=True,
            tokenization=Tokenization.LOWERCASE
        )
    ],

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="content", source_properties=['content','source']
        )
    ]
)

In [8]:
document_objs = list()

for d in documents_text:
    # Extract data from each document
    source = d.metadata['source']
    page = str(d.metadata['page'])  # Convert page number to string if it's an integer
    content = d.page_content

    # Prepare properties for the data object according to your schema
    document_objs.append({
        "source": source,
        "page": page,
        "content": content
    })

# Assuming you have a collection named 'Documents' set up in Weaviate
documents = client.collections.get("citizens_info_docs")
documents.data.insert_many(document_objs)  # Insert all documents at once

BatchObjectReturn(all_responses=[UUID('528165da-54ea-4bee-8e66-287ad368c5e5'), UUID('d0b28834-51ff-4508-8442-2958b07d9a49'), UUID('d6f552b6-2889-4da1-863b-e9e43868dbc3'), UUID('9250580e-7e94-476d-8e43-f4cfe0ef7512'), UUID('953b8865-1401-4f9a-8095-2deb5fc52727'), UUID('b0555e8c-77cf-4b94-82fc-d84d07dda038'), UUID('a13f3c9d-edd1-4b24-b902-0fe626f1a839')], elapsed_seconds=0.5612339973449707, errors={}, uuids={0: UUID('528165da-54ea-4bee-8e66-287ad368c5e5'), 1: UUID('d0b28834-51ff-4508-8442-2958b07d9a49'), 2: UUID('d6f552b6-2889-4da1-863b-e9e43868dbc3'), 3: UUID('9250580e-7e94-476d-8e43-f4cfe0ef7512'), 4: UUID('953b8865-1401-4f9a-8095-2deb5fc52727'), 5: UUID('b0555e8c-77cf-4b94-82fc-d84d07dda038'), 6: UUID('a13f3c9d-edd1-4b24-b902-0fe626f1a839')}, has_errors=False)

In [9]:
query_return = documents.query.fetch_objects(
    include_vector=True, 
    limit=2)

In [10]:
response = documents.query.near_text(
    query="How should one apply for Maternity Benefit and what are the deadlines?",
    limit=1
)

for o in response.objects:
    print(o.properties['content'])  # Object properties

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment
rights including maternity leave.
If you become pregnant while on an employment scheme (Community Employment, Rural Social
Scheme, Tús and Gateway schemes) you are en tled to take 26 weeks’ maternity leave including an
addi onal 16 weeks of op onal unpaid leave. If possible, you will be supported to complete your 52
weeks of paid work on the same or another project when you ﬁnish your maternity leave.
You should discuss with your sponsor the payment you qualify for while you are on maternity leave.
You may qualify for Maternity Beneﬁt if you have enough PRSI contribu ons. Otherwise, if your
spouse or partner is ge ng a social welfare payment, you may get a qualiﬁed adult increase on their
payment.
Page edited: 21 Sep

In [11]:
# Assuming 'client' has been properly initialized and connected to Weaviate
response = documents.query.near_text(
    query="How to apply for social walfare",
    limit=2
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/16/24, 10:04 PM Claiming a social welfare payment
https://www.citizensinformation.ie/en/social-welfare/irish-social-welfare-system/claiming-a-social-welfare-payment/applying-for-a-social-welfare-p… 1/4Claiming a social welfare payment
Introduc on
Rules
How to apply
Where to apply
Further informa on
Introduc on
The Department of Social Protec on has a range of social welfare payments that provide ﬁnancial
support. To get a social welfare payment, you must apply for it. So, if you think you are en tled to a
payment you should always apply.
Your ﬁrst step before applying, is to ﬁnd out which payment you may be en tled to.
Payments are available for:
People who are unemployed
Families and children
Widows, widowers and surviving civil partners
Guardians or orphans
Older and re red people
People with a disability
Carers
If you get a social welfare payment you may also qualify for extra social welfare beneﬁts.
Household Budget Scheme
If you get certain social welfare payments, you can put a

In [26]:
# Assuming 'client' has been properly initialized and connected to Weaviate
documents = client.collections.get("Documents")
response = documents.query.near_text(
    query="What are the rules for switching from Illness Benefit to Maternity Benefit?",
    limit=1
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 2/3What if I am unemployed but don't qualify for a jobseeker's payment because of illness during
pregnancy?
If you are sick or ill during your pregnancy you are not en tled to Jobseeker's Allowance or
Jobseeker's Beneﬁt.
You may get Illness Beneﬁt if you have worked and have enough social insurance contribu ons (see
below). If you are not en tled to Illness Beneﬁt and your illness is expected to last for more than 12
months you may be en tled to Disability Allowance.
If you are pregnant and either not ge ng a social welfare payment or not en tled to any of the
above social welfare payments, you may get some ﬁnancial support under the Supplementary
Welfare Allowance Scheme.
Pregnancy and Illness Beneﬁt
If you are pregnant and claiming Illness Beneﬁt, you will con nue to be paid Illness Beneﬁt if you
have an illnes

In [ ]:
# Assuming 'client' has been properly initialized and connected to Weaviate
documents = client.collections.get("Documents")
response = documents.query.near_text(
    query="Can you participate in employment schemes while pregnant and what rights do you have?",
    limit=1
)

# Printing properties of the objects retrieved
for o in response.objects:
    print(o.properties['content'])  # Object properties

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment
rights including maternity leave.
If you become pregnant while on an employment scheme (Community Employment, Rural Social
Scheme, Tús and Gateway schemes) you are en tled to take 26 weeks’ maternity leave including an
addi onal 16 weeks of op onal unpaid leave. If possible, you will be supported to complete your 52
weeks of paid work on the same or another project when you ﬁnish your maternity leave.
You should discuss with your sponsor the payment you qualify for while you are on maternity leave.
You may qualify for Maternity Beneﬁt if you have enough PRSI contribu ons. Otherwise, if your
spouse or partner is ge ng a social welfare payment, you may get a qualiﬁed adult increase on their
payment.
Page edited: 21 Sep

In [21]:
print(o.properties['content'])

4/12/24, 2:46 PM Pregnancy and social welfare payments
https://www.citizensinformation.ie/en/social-welfare/families-and-children/pregnancy-and-social-welfare-payments/#5011b8 3/3If you are taking part in an employment scheme you have the full range of statutory employment
rights including maternity leave.
If you become pregnant while on an employment scheme (Community Employment, Rural Social
Scheme, Tús and Gateway schemes) you are en tled to take 26 weeks’ maternity leave including an
addi onal 16 weeks of op onal unpaid leave. If possible, you will be supported to complete your 52
weeks of paid work on the same or another project when you ﬁnish your maternity leave.
You should discuss with your sponsor the payment you qualify for while you are on maternity leave.
You may qualify for Maternity Beneﬁt if you have enough PRSI contribu ons. Otherwise, if your
spouse or partner is ge ng a social welfare payment, you may get a qualiﬁed adult increase on their
payment.
Page edited: 21 Sep